# Filtrado de datos
Se eliminan duplicados, filas sin sinópisis y/o sin géneros. Luego se seleccionan los 18 géneros más comunes.

In [ ]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Loading dataset
dataset = load_dataset("pkchwy/letterboxd-all-movie-data")

train = dataset['train']

df = pd.json_normalize(train)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

full_dump.jsonl:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/847209 [00:00<?, ? examples/s]

In [ ]:
# filter all
filtered_df = df.drop_duplicates(subset=['title'])

zero_genres = filtered_df['genres'].apply(lambda x: len(x) if isinstance(x, (list, tuple)) else 0).eq(0)
filtered_df = filtered_df[~zero_genres]

zero = filtered_df['synopsis'].apply(lambda x: len(x) if isinstance(x, str) else 0).eq(0)
filtered_df = filtered_df[~zero]

top18 = [
    "Drama", "Documentary", "Comedy", "Animation",
    "Horror", "Romance", "Thriller", "Music",
    "Action", "Crime", "Family", "Adventure",
    "Fantasy", "Science Fiction", "Mistery", "History",
    "War", "Western"
]

mask = filtered_df["genres"].apply(lambda g: all(genre in top18 for genre in g))
filtered_df = filtered_df[mask].copy()
filtered_df = filtered_df[["synopsis", "genres"]]

# Creación de datasets final (con variedad de clases)
Se usará el dataset One-genre: solo películas asignadas a un único género.

A su vez se utilizarán solo N sinopsis de cada género.





## One Genre

In [ ]:
one_genre_df = filtered_df[filtered_df["genres"].apply(lambda x: len(x) == 1)]
unique_genres = one_genre_df["genres"].apply(lambda x: x[0]).unique()

for genre in unique_genres:
  count = len(one_genre_df[one_genre_df["genres"].apply(lambda x: x[0] == genre)])
  if count < 1500:
    one_genre_df = one_genre_df[one_genre_df["genres"].apply(lambda x: x[0] != genre)]

unique_genres = one_genre_df["genres"].apply(lambda x: x[0]).unique()

selected_films = {}
for genre in unique_genres:
  # trimmed change to 1500 for final run
  selected_films[genre] = one_genre_df[one_genre_df["genres"].apply(lambda x: x[0] == genre)].sort_values(by="synopsis", ascending=False)
  selected_films[genre]["genre"] = genre
  selected_films[genre].drop(columns=["genres"], inplace=True)

one_genre_df = pd.concat(selected_films.values())

# Modelo NER-BERT
Se carga el modelo y se le aplica la tarea al dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import pandas as pd
import numpy as np # Used for placeholder value
from typing import List, Dict, Any

# Define the maximum number of items to process per genre
MAX_SYNOPSES = 1000

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# NER per genre
for genre in unique_genres:
    full_synopsis_list: List[str] = selected_films[genre]["synopsis"].tolist()

    total_synopses: int = len(full_synopsis_list)

    synopses_to_process: List[str] = full_synopsis_list[:MAX_SYNOPSES]

    print(f"Processing {len(synopses_to_process)} synopses for genre: {genre}...")
    ner_results_slice: List[List[Dict[str, Any]]] = nlp(synopses_to_process)

    full_ner_results: List[Any] = [None] * total_synopses

    full_ner_results[:MAX_SYNOPSES] = ner_results_slice

    selected_films[genre]["ner_results"] = full_ner_results

    print(f"Successfully saved {len(ner_results_slice)} results. Remaining {total_synopses - len(ner_results_slice)} entries are set to 'None'.")

Device set to use cpu


Processing 1000 synopses for genre: Drama...
Successfully saved 1000 results. Remaining 63626 entries are set to 'None'.
Processing 1000 synopses for genre: Documentary...


KeyboardInterrupt: 

## NER
Recordemos las entidades con las que vamos a trabajar
| Entidad| Significado |
| ------ | ----------- |
| O      | Entidad no nombrada|
| B-MISC | Comienzo de una entidad miscelánea que le sigue a otra entidad miscelanea|
| I-MISC | Entidad miscelánea |
| B-PER  | Comienzo del nombre de una persona justo después del nombre de otra persona |
| I-PER  | Nombre de persona |
| B-ORG  | Comienzo de una organización justo después de otra organización |
| I-ORG  | Organización |
| B-LOC  | Comienzo de una ubicación justo después de otra ubicación |
| I-LOC  | ubicación |


In [ ]:
from collections import Counter

CONFIDENCE_THRESHOLD = 0.8
simplified_entities = ["MISC", "PER", "ORG", "LOC"]

analysis_per_genre = {}

for genre in unique_genres:
    ner_results = selected_films[genre]["ner_results"]
    words_per_entity = {}

    for synopsis in ner_results:
        if synopsis is None:
            continue

        current_entity_word = ""
        current_entity_type = None

        for entity in synopsis:
            if entity is None:
                continue

            if entity.get("score", 0.0) < CONFIDENCE_THRESHOLD:

                if current_entity_word and current_entity_type:
                    # Save the previously built entity before resetting
                    if current_entity_type not in words_per_entity:
                        words_per_entity[current_entity_type] = []
                    words_per_entity[current_entity_type].append(current_entity_word)

                # Reset the buffer
                current_entity_word = ""
                current_entity_type = None
                continue

            # 2. Get Tag Information
            tag = entity["entity"]
            word = entity["word"]

            # --- Check for the end of an entity sequence ---

            if tag.startswith("B-") or tag == "O" or not current_entity_word:
                # If we encounter a new entity (B-XXX), 'O', or it's the start of the text:

                # Save the PREVIOUS entity if one was being built
                if current_entity_word and current_entity_type:
                    if current_entity_type not in words_per_entity:
                        words_per_entity[current_entity_type] = []
                    words_per_entity[current_entity_type].append(current_entity_word)

                # Reset and start the new entity sequence
                current_entity_word = ""
                current_entity_type = None

                if tag.startswith("B-"):
                    # Start the new entity
                    current_entity_type = tag.split('-')[-1]

                    # Clean the word: remove '##' and add space
                    if word.startswith('##'):
                        current_entity_word = word.lstrip('##')
                    else:
                        current_entity_word = word

            # --- Check for continuation of an entity sequence ---

            elif tag.startswith("I-") and tag.split('-')[-1] == current_entity_type:
                # Continuation token for the current entity type

                # Concatenate with a space or no space based on '##'
                if word.startswith('##'):
                    # No space if it's a sub-word token fragment
                    current_entity_word += word.lstrip('##')
                else:
                    # Add a space for whole word continuation (e.g., "East" + "Berlin")
                    current_entity_word += " " + word

            # Note: Tags like 'O' are handled by the 'B-' logic above, resetting the buffer.

        # --- AFTER LOOP: Handle the last entity in the synopsis ---
        if current_entity_word and current_entity_type:
            if current_entity_type not in words_per_entity:
                words_per_entity[current_entity_type] = []
            words_per_entity[current_entity_type].append(current_entity_word)

    selected_films[genre]["words_per_entity"] = words_per_entity

    genre_analysis = {}
    for entity_type in simplified_entities:
        if entity_type in words_per_entity:
            words = words_per_entity[entity_type]
            word_counts = Counter(words)
            ranked_entities = word_counts.most_common()
            genre_analysis[entity_type] = ranked_entities

            print(f"🎬 {genre} - {entity_type}: Top 5: {ranked_entities[:5]}")

    analysis_per_genre[genre] = genre_analysis

🎬 Drama - MISC: Top 5: [('French', 11), ('American', 9), ('Japanese', 8), ('Brazilian', 6), ('Chinese', 5)]
🎬 Drama - PER: Top 5: [('Z', 184), ('Yu', 167), ('Yo', 74), ('ko', 44), ('A', 34)]
🎬 Drama - ORG: Top 5: [('Reşit', 4), ('BFI', 2), ('İsmail', 2), ('SM', 2), ('S & M', 2)]
🎬 Drama - LOC: Top 5: [('Tokyo', 14), ('Paris', 12), ('France', 10), ('Berlin', 8), ('Japan', 7)]
🎬 Documentary - MISC: Top 5: [('American', 156), ('French', 67), ('German', 56), ('Japanese', 48), ('Chinese', 47)]
🎬 Documentary - PER: Top 5: [('Z', 108), ('G', 68), ('A', 62), ('V', 62), ('W', 62)]
🎬 Documentary - ORG: Top 5: [('Edison', 11), ('B', 9), ('UN', 9), ('WWE', 9), ('Google', 9)]
🎬 Documentary - LOC: Top 5: [('America', 111), ('Europe', 83), ('Russia', 56), ('United States', 55), ('Canada', 53)]
🎬 Animation - MISC: Top 5: [('Japanese', 59), ('American', 32), ('French', 23), ('C', 21), ('Chinese', 20)]
🎬 Animation - PER: Top 5: [('G', 71), ('Woody', 64), ('A', 52), ('T', 52), ('P', 46)]
🎬 Animation - OR

# TF-IdF


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def extract_top_keywords_by_genre(selected_films, top_n=5):
    """
    Calculates TF-IDF for each genre's synopses and extracts the top N keywords.

    :param selected_films: Dictionary where keys are genres and values are lists of synopses.
    :param top_n: The number of top words to return for each genre.
    :return: A dictionary of genres mapping to their top keywords and scores.
    """
    genre_keywords = {}

    for genre, synopses_list in selected_films.items():
        # Combine all synopses for the current genre into a single document (string)
        # This treats the *entire collection of synopses for one genre* as the corpus/document
        # This approach helps find words important *to that genre* relative to all words in the genre

        corpus = [" ".join(synopses_list["synopsis"])]

        # Initialize TfidfVectorizer
        # 'english' stop words removes common terms like 'the', 'a', 'is', etc.
        # token_pattern is adjusted to capture words of 2 or more letters.
        vectorizer = TfidfVectorizer(
            stop_words='english',
            token_pattern=r'\b[a-z]{2,}\b'
        )

        tfidf_matrix = vectorizer.fit_transform(corpus)

        # Get feature names (words)
        feature_names = vectorizer.get_feature_names_out()

        # Get the TF-IDF scores for the first (and only) document (the combined genre text)
        tfidf_scores = tfidf_matrix.toarray()[0]

        # Create a pandas Series for easy sorting
        feature_series = pd.Series(tfidf_scores, index=feature_names)

        # Sort the words by their TF-IDF score in descending order
        sorted_keywords = feature_series.sort_values(ascending=False)

        # Select the top N keywords and their scores
        top_keywords = sorted_keywords.head(top_n).to_dict()

        genre_keywords[genre] = top_keywords

    return genre_keywords

top_words = extract_top_keywords_by_genre(selected_films, top_n=5)

for genre, keywords in top_words.items():
    print(f"## 🎬 Top {len(keywords)} Keywords for {genre}:")
    for word, score in keywords.items():
        print(f"- **{word}**: {score:.4f}")
    print("---")

## 🎬 Top 5 Keywords for Drama:
- **life**: 0.3078
- **young**: 0.2697
- **film**: 0.2040
- **man**: 0.2036
- **love**: 0.1986
---
## 🎬 Top 5 Keywords for Documentary:
- **film**: 0.4482
- **documentary**: 0.3525
- **life**: 0.2574
- **world**: 0.2179
- **years**: 0.1624
---
## 🎬 Top 5 Keywords for Animation:
- **film**: 0.3926
- **short**: 0.2339
- **animated**: 0.2136
- **animation**: 0.2013
- **world**: 0.1994
---
## 🎬 Top 5 Keywords for Music:
- **live**: 0.4512
- **concert**: 0.3039
- **band**: 0.2355
- **music**: 0.2275
- **album**: 0.1802
---
## 🎬 Top 5 Keywords for Comedy:
- **comedy**: 0.2369
- **life**: 0.2228
- **man**: 0.2155
- **film**: 0.1854
- **young**: 0.1778
---
## 🎬 Top 5 Keywords for Western:
- **gang**: 0.2574
- **town**: 0.2442
- **ranch**: 0.1987
- **man**: 0.1913
- **sheriff**: 0.1897
---
## 🎬 Top 5 Keywords for Horror:
- **horror**: 0.2878
- **film**: 0.2505
- **young**: 0.2036
- **man**: 0.1781
- **woman**: 0.1759
---
## 🎬 Top 5 Keywords for Science Fiction:
- 

# Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

def extract_top_frequent_words_by_genre(selected_films, top_n=5):
    """
    Calculates Bag-of-Words (word counts) for each genre's synopses
    and extracts the top N most frequent words.

    :param selected_films: Dictionary where keys are genres and values are lists of synopses.
    :param top_n: The number of top words to return for each genre.
    :return: A dictionary of genres mapping to their top words and counts.
    """
    genre_keywords = {}

    for genre, synopses_list in selected_films.items():
        # Combine all synopses for the current genre into a single document (string)
        corpus = [" ".join(synopses_list["synopsis"])]

        # Initialize CountVectorizer
        # 'english' stop words removes common terms like 'the', 'a', 'is', etc.
        # token_pattern is adjusted to capture words of 2 or more letters.
        vectorizer = CountVectorizer(
            stop_words='english',
            token_pattern=r'\b[a-z]{2,}\b'
        )

        # Fit and transform the corpus (creates the Bag-of-Words matrix)
        bow_matrix = vectorizer.fit_transform(corpus)

        # Get feature names (words)
        feature_names = vectorizer.get_feature_names_out()

        # Get the word counts for the first (and only) document (the combined genre text)
        word_counts = bow_matrix.toarray()[0]

        # Create a pandas Series for easy sorting
        feature_series = pd.Series(word_counts, index=feature_names)

        # Sort the words by their count in descending order
        sorted_keywords = feature_series.sort_values(ascending=False)

        # Select the top N keywords and their counts
        top_keywords = sorted_keywords.head(top_n).to_dict()

        genre_keywords[genre] = top_keywords

    return genre_keywords

# --- Run the function with your data ---
top_words = extract_top_frequent_words_by_genre(selected_films, top_n=5)

# --- Display the results ---
for genre, keywords in top_words.items():
    print(f"## 📝 Top {len(keywords)} Most Frequent Words for {genre}:")
    for word, count in keywords.items():
        print(f"- **{word}**: {count} occurrences")
    print("---")

## 📝 Top 5 Most Frequent Words for Drama:
- **life**: 13873 occurrences
- **young**: 12156 occurrences
- **film**: 9194 occurrences
- **man**: 9177 occurrences
- **love**: 8951 occurrences
---
## 📝 Top 5 Most Frequent Words for Documentary:
- **film**: 30011 occurrences
- **documentary**: 23604 occurrences
- **life**: 17231 occurrences
- **world**: 14589 occurrences
- **years**: 10875 occurrences
---
## 📝 Top 5 Most Frequent Words for Animation:
- **film**: 3856 occurrences
- **short**: 2297 occurrences
- **animated**: 2098 occurrences
- **animation**: 1977 occurrences
- **world**: 1958 occurrences
---
## 📝 Top 5 Most Frequent Words for Music:
- **live**: 5109 occurrences
- **concert**: 3442 occurrences
- **band**: 2667 occurrences
- **music**: 2576 occurrences
- **album**: 2041 occurrences
---
## 📝 Top 5 Most Frequent Words for Comedy:
- **comedy**: 4847 occurrences
- **life**: 4558 occurrences
- **man**: 4408 occurrences
- **film**: 3792 occurrences
- **young**: 3638 occurrences
---
